In [1]:
import pandas as pd
import seaborn as sns

In [2]:
datadir = '../data/'

In [3]:
def _read_file(fname, enc='iso8859-8'):
    fd = open(fname, encoding=enc, errors='replace')
    df = pd.read_csv(fd, sep='|')
    return df

def add_model(df):
    df['model'] = df.apply(lambda x: '_'.join([x[y] for y in ['tozeret_cd', 'degem_cd', 'shnat_yitzur', 'sug_degem']]), axis=1)

def get_model_name(ns):
    names = pd.DataFrame(data={'model':ns})
    ret = pd.merge(names, models, how='left', on='model')[['tozeret_nm', 'kinuy_mishari']]
    return ret

In [4]:
# source : https://data.gov.il/dataset/private-and-commercial-vehicles

#df = pd.read_csv('data/rechev-small.csv', encoding='iso-8859-1', sep='|')

def read_file(fname):
    fname = datadir + fname
    df = _read_file(fname)

    for c in ['mispar_rechev', 'degem_cd', 'tozeret_cd', 'shnat_yitzur']:
        df[c] = df[c].astype(str)

    df['test']= pd.to_datetime(df.mivchan_acharon_dt)
    df['test_expiry']= pd.to_datetime(df.tokef_dt)    
    add_model(df)
    return df

In [5]:
prev = read_file('rechev-20230401.csv')
curr = read_file('rechev-20230501.csv')

In [6]:
# perform left and right joins
df_prev = pd.merge(prev, curr['mispar_rechev'], on='mispar_rechev', how='left')
df_prev['src'] = 'prev'
df_curr = pd.merge(prev['mispar_rechev'], curr, on='mispar_rechev', how='right')
df_curr['src'] = 'curr'
df = pd.concat([df_prev, df_curr])
df.drop_duplicates(subset='mispar_rechev', keep=False, inplace=True)

In [7]:
df['src'].value_counts()

curr    18482
prev    15349
Name: src, dtype: int64

In [8]:
df.query('src == "prev"')['shnat_yitzur'].value_counts()

2008    1351
2011    1017
2012     825
2010     818
2007     757
2006     742
2016     731
2015     728
2019     722
2017     698
2009     691
2014     681
2018     678
2013     654
2021     636
2005     628
2003     585
2004     480
2002     420
2001     368
2022     308
2020     303
2000     189
1999     138
1998     103
1997      48
2023      45
1996       5
Name: shnat_yitzur, dtype: int64

In [9]:
df.query('src == "curr"')['shnat_yitzur'].value_counts()

2023    15490
2016      215
2018      200
2015      191
2020      189
2017      187
2011      186
2019      184
2021      183
2008      179
2012      155
2013      146
2010      141
2014      135
2009      123
2007       99
2003       91
2006       87
2002       80
2005       55
2004       39
2001       36
2000       28
2022       19
1998       16
1999       15
1997       12
1996        1
Name: shnat_yitzur, dtype: int64

In [27]:
onlycurr = df.query('src == "curr"').copy()

In [28]:
onlycurr[onlycurr.sug_delek_nm.str.contains('חשמל')].sug_delek_nm.value_counts()

חשמל          3021
חשמל/בנזין    1007
חשמל/דיזל        8
Name: sug_delek_nm, dtype: int64

In [29]:
electric = onlycurr[onlycurr.sug_delek_nm.str.contains('חשמל')].copy()

In [32]:
electric.kinuy_mishari.value_counts().head(20)

ATTO 3            1418
NIRO PLUS PHEV     473
GEOMETRY C         300
ORA FUNKY CAT      110
ID4                110
ENYAQ IV           103
EHS PHEV            95
TOYOTA BZ4X         93
U5                  88
IONIQ5              76
NIRO PHEV           67
SERES 3             65
EQA250              60
ET5                 48
MODEL Y             46
T03                 44
WEY COFFEE 01       42
MODEL X             41
SPORTAGE            38
LYNKCO 01           32
Name: kinuy_mishari, dtype: int64